In [192]:
!pip install langid
!pip install -U sentence-transformers


Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.2.5)


In [193]:
!git clone https://github.com/LucaBassanese/Tesi

fatal: destination path 'Tesi' already exists and is not an empty directory.


In [194]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy


import matplotlib.pyplot as plt

#Keywords
import sklearn

#wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from nltk.corpus import stopwords

#transalte
import langid

import random


#cluster of papers
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#cluster hierarchical
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import ward, dendrogram

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
pd.options.display.max_rows = 150

#SBERT
from sentence_transformers import SentenceTransformer

#cosine similarity
from scipy import spatial

#combinazioni
from itertools import combinations 


from nltk.util import ngrams

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Titolo e Abstract

In [0]:
url =  'https://raw.githubusercontent.com/LucaBassanese/Tesi/master/data/abstract.xlsx'
df = pd.read_excel(url)
dft = df.copy()
df = df[df['Abstract inglese'].notnull()]
df = df.drop_duplicates(['Abstract inglese'])
df['titabs']=  df.Titolo + ' ' + df['Abstract inglese']

lingua = [langid.classify(testo)[0] for testo in df.Titolo]
lingua = pd.Series(lingua)
df = df[(lingua == 'en').values]

#tokenizer che toglie la punteggiatura
tokenizer = RegexpTokenizer(r'\w+')


In [0]:
#tokenizzo togliendo la punteggiatura
words_ta = [tokenizer.tokenize(fr.lower()) for fr in df.titabs] 
#creo bigrammi e trigrammi
bigrams_t = [list(ngrams(word,2)) for word in words_ta ]
bigrams_ta = [['_'.join(list(w)) for w in bi] for bi in bigrams_t]
trigrams_t = [list(ngrams(word,3)) for word in words_ta ]
trigrams_ta = [['_'.join(list(w)) for w in tri] for tri in trigrams_t]

In [0]:
#creo i tagged document per il doc2vec
tagged_data_ta = [TaggedDocument(words=words_ta[i], tags=[str(i)]) for i in range(len(df))]
tagged_bi_ta = [TaggedDocument(words=bigrams_ta[i], tags=[str(i)]) for i in range(len(df))]
tagged_tri_ta = [TaggedDocument(words=trigrams_ta[i], tags=[str(i)]) for i in range(len(df))]

# Titoli


In [0]:
dft = dft.drop_duplicates(['Titolo'])
# Seleziono sogli i titoli in inglese
lingua = [langid.classify(testo)[0] for testo in dft.Titolo]
lingua = pd.Series(lingua)
dft = dft[(lingua == 'en').values]

In [199]:
#tokenizzo togliendo la punteggiatura
words = [tokenizer.tokenize(fr.lower()) for fr in dft.Titolo ] 
#creo bigrammi e trigrammi
bigrams_t = [list(ngrams(word,2)) for word in words ]
bigrams = [['_'.join(list(w)) for w in bi] for bi in bigrams_t]
trigrams_t = [list(ngrams(word,3)) for word in words ]
trigrams = [['_'.join(list(w)) for w in tri] for tri in trigrams_t]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: generator 'ngrams' raised StopIteration
  """


In [0]:
#creo i tagged document per il doc2vec
tagged_data = [TaggedDocument(words=words[i], tags=[str(i)]) for i in range(len(dft))]
tagged_bi = [TaggedDocument(words=bigrams[i], tags=[str(i)]) for i in range(len(dft))]
tagged_tri = [TaggedDocument(words=trigrams[i], tags=[str(i)]) for i in range(len(dft))]


# Frasi unite

In [0]:
frasi = [' '.join(word) for word in words]
frasi_bi = [' '.join(bi) for bi in bigrams]
frasi_tri = [' '.join(tri) for tri in trigrams]
frasi_ta =  [' '.join(word) for word in words_ta]
frasi_bi_ta = [' '.join(bi) for bi in bigrams_ta]
frasi_tri_ta = [' '.join(tri) for tri in trigrams_ta]

# Distribuzione parole 


In [0]:
#creo lessico di tutte le parole
lessico = tokenizer.tokenize(' '.join(frasi ))
stop_words = set(stopwords.words('english')) 
lessico = [w for w in lessico if not w in stop_words] 
#creo la distribuzione delle parole nei titoli
distr = (pd.Series(lessico)).value_counts()
#elimino le parole troppo rare
distr = distr[distr >2] 

In [0]:
parover95 = list((distr[distr > np.percentile(distr, 95)]).index)
par75_80 = list((distr[distr.between(np.percentile(distr, 75), np.percentile(distr, 80))]).index)
par50_55 = list((distr[distr.between(np.percentile(distr, 50), np.percentile(distr, 55))]).index)

In [0]:
#indici per 95 percentile
parindoc_1 = [[d.count(wr) for wr in parover95]for d in frasi]
somma_1 = [sum(doc) for doc in parindoc_1]
ind_1 = sorted(range(len(somma_1)), key=lambda i: somma_1[i])[-10:]

#indici per parole tra il 75 e 80
parindoc_2 = [[d.count(wr) for wr in par75_80]for d in frasi]
somma_2 = [sum(doc) for doc in parindoc_2]
ind_2 = sorted(range(len(somma_2)), key=lambda i: somma_2[i])[-10:]

# indici per parole tra 50 e 55
parindoc_3 = [[d.count(wr) for wr in par50_55]for d in frasi]
somma_3 = [sum(doc) for doc in parindoc_3]
ind_3 = sorted(range(len(somma_3)), key=lambda i: somma_3[i])[-11:]

#creo indici
indici = (pd.Series(ind_1+ ind_2 + ind_3)).drop_duplicates()


# Cosine similarity documenti scelti

## Hs

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_hs_300.csv')

In [0]:
comb = combinations(indici.values, 2)
sims = []
ind_comb = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  # print(f'First: {i[0]}, Second: {i[1]},\
  # Similarity: {sim}')
  sims.append( sim)
  ind_comb.append(str(i[0]) + ' - ' + str(i[1]))

In [0]:
matriciona = pd.DataFrame(columns= ind_comb)

In [0]:
matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)

In [0]:
ind_mod = ['sent2vec_uni_hs_300']

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_hs_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_hs_400') 

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_hs_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_hs_500') 

## ns

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_ns_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_ns_300') 

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_ns_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_ns_400') 

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_ns_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_ns_500') 

## Softmax

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_softmax_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_softmax_300') 

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_softmax_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_softmax_400') 

In [0]:
a = pd.read_csv('/content/Tesi/data/sent2vec_uni_softmax_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('sent2vec_uni_softmax_500') 

In [0]:
matriciona.index = ind_mod

In [219]:
matriciona

,781 - 833,781 - 838,781 - 844,781 - 877,781 - 892,781 - 1326,781 - 103,781 - 842,781 - 1107,781 - 1356,781 - 1384,781 - 1395,781 - 1402,781 - 1404,781 - 1405,781 - 1475,781 - 0,781 - 187,781 - 504,781 - 1117,781 - 1124,781 - 590,781 - 787,781 - 1118,781 - 1205,781 - 634,781 - 952,781 - 445,781 - 587,833 - 838,833 - 844,833 - 877,833 - 892,833 - 1326,833 - 103,833 - 842,833 - 1107,833 - 1356,833 - 1384,833 - 1395,...,1117 - 634,1117 - 952,1117 - 445,1117 - 587,1124 - 590,1124 - 787,1124 - 1118,1124 - 1205,1124 - 634,1124 - 952,1124 - 445,1124 - 587,590 - 787,590 - 1118,590 - 1205,590 - 634,590 - 952,590 - 445,590 - 587,787 - 1118,787 - 1205,787 - 634,787 - 952,787 - 445,787 - 587,1118 - 1205,1118 - 634,1118 - 952,1118 - 445,1118 - 587,1205 - 634,1205 - 952,1205 - 445,1205 - 587,634 - 952,634 - 445,634 - 587,952 - 445,952 - 587,445 - 587
sent2vec_uni_hs_300,0.22,0.35,0.94,0.26,0.97,0.35,-0.13,0.35,0.59,0.49,-0.26,-0.05,0.54,0.78,0.72,-0.01,0.12,0.86,0.85,0.73,0.71,0.49,0.93,0.93,-0.02,0.85,0.71,0.90,0.81,0.32,0.02,0.63,0.30,0.32,0.72,0.87,0.61,0.88,0.56,0.77,...,0.74,0.77,0.76,0.86,0.89,0.86,0.85,0.61,0.75,0.82,0.55,0.83,0.73,0.68,0.80,0.56,0.84,0.42,0.76,0.99,0.23,0.90,0.84,0.88,0.94,0.16,0.93,0.80,0.89,0.93,0.05,0.50,-0.18,0.26,0.79,0.88,0.93,0.69,0.90,0.85
sent2vec_uni_hs_400,0.28,0.43,0.94,0.31,0.98,0.43,-0.05,0.37,0.62,0.51,-0.20,0.02,0.57,0.79,0.73,0.05,0.19,0.88,0.86,0.77,0.73,0.55,0.93,0.93,0.05,0.85,0.74,0.90,0.82,0.40,0.08,0.74,0.37,0.40,0.76,0.90,0.66,0.89,0.58,0.82,...,0.79,0.81,0.78,0.90,0.90,0.87,0.86,0.64,0.77,0.84,0.55,0.85,0.76,0.72,0.80,0.62,0.87,0.45,0.80,0.99,0.27,0.92,0.87,0.88,0.95,0.22,0.94,0.83,0.89,0.94,0.12,0.52,-0.15,0.31,0.82,0.89,0.94,0.70,0.91,0.85
sent2vec_uni_hs_500,0.26,0.43,0.95,0.34,0.98,0.43,-0.09,0.34,0.66,0.48,-0.16,0.05,0.58,0.81,0.73,0.08,0.21,0.88,0.87,0.76,0.72,0.53,0.93,0.93,0.04,0.87,0.78,0.91,0.83,0.43,0.06,0.77,0.36,0.43,0.75,0.92,0.66,0.91,0.63,0.84,...,0.77,0.80,0.75,0.88,0.91,0.87,0.86,0.65,0.77,0.84,0.55,0.84,0.76,0.72,0.81,0.61,0.83,0.43,0.77,0.99,0.28,0.92,0.88,0.88,0.95,0.23,0.94,0.85,0.89,0.95,0.11,0.48,-0.15,0.29,0.84,0.89,0.95,0.73,0.92,0.86
sent2vec_uni_ns_300,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
sent2vec_uni_ns_400,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
sent2vec_uni_ns_500,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
sent2vec_uni_softmax_300,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,0.99,1.00,1.00,1.00,0.99,0.99,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,0.99,1.00,1.00,1.00,1.00,1.00,1.00,1.00
sent2vec_uni_softmax_400,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1

## Doc2vec uni

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_uni_dm_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_uni_dm_300') 

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_uni_dm_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_uni_dm_400') 

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_uni_dm_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_uni_dm_500') 

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_uni_dbow_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_uni_dbow_300') 

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_uni_dbow_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_uni_dbow_400') 

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_uni_dbow_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_uni_dbow_500') 

## Doc2vec Bi

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_bi_dm_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_bi_dm_300')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_bi_dm_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_bi_dm_400')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_bi_dm_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_bi_dm_500')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_bi_dbow_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_bi_dbow_300')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_bi_dbow_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_bi_dbow_400')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_bi_dbow_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_bi_dbow_500')

## Doc2vec tri

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_tri_dm_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_tri_dm_300')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_tri_dm_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_tri_dm_400')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_tri_dm_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_tri_dm_500')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_tri_dbow_300.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_tri_dbow_300')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_tri_dbow_400.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_tri_dbow_400')

In [0]:
a = pd.read_csv('/content/Tesi/data/doc2vec_tri_dbow_500.csv')
#per tutti i modelli
comb = combinations(indici.values, 2)
sims = []
for i in list(comb):
  sim = round(1-spatial.distance.cosine(a.iloc[i[0]], a.iloc[i[1]]),2)
  sims.append( sim)

matriciona = matriciona.append(pd.Series(sims, index=matriciona.columns ), ignore_index= True)
ind_mod.append('doc2vec_tri_dbow_500')

In [0]:
matriciona.index = ind_mod

In [0]:
matriciona.to_excel('matriciona.xlsx')

# Frasi to txt

In [0]:
pd.DataFrame(frasi, columns=['frasi']).to_csv('frasi.txt',  index=None, header = False)
pd.DataFrame(frasi_bi, columns=['frasi']).to_csv('frasi_bi.txt',  index=None, header = False)
pd.DataFrame(frasi_tri, columns=['frasi']).to_csv('frasi_tri.txt',  index=None, header = False)

# Doc2vec


In [0]:
# esempio 
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha




## Data dm


In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_uni_dm_300.csv', index= False)

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_uni_dm_400.csv', index= False)

In [0]:
#model.save('doc2vec_uni_dm_400')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_uni_dm_500.csv', index= False)
    

#model.save('doc2vec_uni_dm_500')


## Data dbow

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_uni_dbow_300.csv', index= False)
    

#model.save('doc2vec_uni_dbow_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_uni_dbow_400.csv', index= False)
#model.save('doc2vec_uni_dbow_400')

In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_uni_dbow_500.csv', index= False)
#model.save('doc2vec_uni_dbow_500')

In [0]:
a


## Bigram dm

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_bi_dm_300.csv', index= False)
#model.save('doc2vec_bi_dm_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_bi_dm_400.csv', index= False)
#model.save('doc2vec_bi_dm_400')

In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_bi_dm_500.csv', index= False)
#model.save('doc2vec_bi_dm_500')

## Bigram dbow

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_bi_dbow_300.csv', index= False)
#model.save('doc2vec_bi_dbow_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_bi_dbow_400.csv', index= False)
#model.save('doc2vec_bi_dbow_400')

In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_bi)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_bi,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_bi_dbow_500.csv', index= False)
#model.save('doc2vec_bi_dbow_500')

## Trigram dm

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_tri_dm_300.csv', index= False)
#model.save('doc2vec_tri_dm_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_tri_dm_400.csv', index= False)
#model.save('doc2vec_tri_dm_400')

In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                #dbow_words = 1
                dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_tri_dm_500.csv', index= False)
#model.save('doc2vec_tri_dm_500')

## Trigram dbow

In [0]:
%%time
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    

matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_tri_dbow_300.csv', index= False)
#model.save('doc2vec_tri_dbow_300')

In [0]:
%%time
max_epochs = 100
vec_size = 400
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_tri_dbow_400.csv', index= False)
#model.save('doc2vec_tri_dbow_400')

In [0]:
%%time
max_epochs = 100
vec_size = 500
alpha = 0.025

model = Doc2Vec(vector_size = vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                window = 3,
                #dm = 0,
                dbow_words = 1
                #dm_concat = 1
                )
  
model.build_vocab(tagged_tri)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_tri,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('/content/Tesi/data/doc2vec_tri_dbow_500.csv', index= False)
#model.save('doc2vec_tri_dbow_500')


# Load doc2vec

## Uni

In [0]:
model = Doc2Vec.load('doc2vec_uni_dbow_300')
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_uni_dbow_300.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_uni_dbow_400')
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_uni_dbow_400.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_uni_dbow_500')
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_uni_dbow_500.csv', index= False)


In [0]:
model = Doc2Vec.load('doc2vec_uni_dm_300')
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_uni_dm_300.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_uni_dm_400')
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_uni_dm_400.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_uni_dm_500')
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_uni_dm_500.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

## Bi

In [0]:
model = Doc2Vec.load('doc2vec_bi_dbow_300')
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_bi_dbow_300.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_bi_dbow_400')
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_bi_dbow_400.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_bi_dbow_500')
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_bi_dbow_500.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_bi_dm_300')
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_bi_dm_300.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
model = Doc2Vec.load('doc2vec_bi_dm_400')
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_bi_dm_400.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
model = Doc2Vec.load('doc2vec_bi_dm_500')
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_bi_dm_500.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

## Tri

In [0]:
model = Doc2Vec.load('doc2vec_tri_dm_300')
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_tri_dm_300.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
model = Doc2Vec.load('doc2vec_tri_dm_400')
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_tri_dm_400.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
model = Doc2Vec.load('doc2vec_tri_dm_500')
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_tri_dm_500.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
model = Doc2Vec.load('doc2vec_tri_dbow_300')
matrix = np.zeros((len(dft),300))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_tri_dbow_300.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_tri_dbow_400')
matrix = np.zeros((len(dft),400))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_tri_dbow_400.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model = Doc2Vec.load('doc2vec_tri_dbow_500')
matrix = np.zeros((len(dft),500))
for i in range(len(dft)):
  matrix[i,:] = model.docvecs[i]

a = pd.DataFrame(matrix)
a.to_csv('doc2vec_tri_dbow_500.csv', index= False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# SBERT

In [0]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(frasi)
sbert = np.stack( sentence_embeddings, axis=0 )

100%|██████████| 405M/405M [00:24<00:00, 16.4MB/s]


In [0]:
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

100%|██████████| 1.24G/1.24G [00:18<00:00, 67.1MB/s]


In [0]:
import pandas as pd
pd.read_csv??

In [0]:
sentence_embeddings = model.encode(frasi)
sbert = np.stack( sentence_embeddings, axis=0 )

In [0]:
sbert.shape

(1548, 1024)

In [0]:
sbert

array([[-1.1434323 , -0.18188004,  0.91623545, ..., -0.48097208,
        -0.9856035 ,  0.8776383 ],
       [-0.40389788, -0.41126215,  0.42457682, ...,  0.5315726 ,
        -0.87461996,  0.44353035],
       [-0.24677515, -0.12850131,  0.44034338, ..., -0.7237369 ,
        -0.31194127, -0.29219583],
       ...,
       [-0.3599878 ,  0.2995323 ,  0.62660736, ..., -0.2494359 ,
        -0.6457086 ,  0.389404  ],
       [-0.49856946,  0.05594404,  0.3238621 , ..., -0.1141041 ,
        -0.24745905,  0.60181206],
       [ 0.03410498, -0.08580752,  1.8610344 , ..., -1.1958299 ,
        -1.1618495 , -0.1159507 ]], dtype=float32)